In [3]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [5]:
prod_json = pd.read_csv('../prod_json.json')

In [5]:
index = [value for value in range(44099)]

In [6]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
df['index'] = index

In [8]:
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image,filename,index
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,http://assets.myntassets.com/v1/images/style/p...,15970.jpg,0
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,http://assets.myntassets.com/v1/images/style/p...,39386.jpg,1
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,http://assets.myntassets.com/v1/images/style/p...,59263.jpg,2
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,http://assets.myntassets.com/v1/images/style/p...,21379.jpg,3
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,http://assets.myntassets.com/v1/images/style/p...,53759.jpg,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44094,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013,Casual,Gas Men Caddy Casual Shoe,http://assets.myntassets.com/v1/images/style/p...,17036.jpg,44094
44095,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011,Casual,Lotto Men's Soccer Track Flip Flop,http://assets.myntassets.com/v1/images/style/p...,6461.jpg,44095
44096,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011,Casual,Puma Men Graphic Stellar Blue Tshirt,http://assets.myntassets.com/v1/images/style/p...,18842.jpg,44096
44097,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017,Casual,Rasasi Women Blue Lady Perfume,http://assets.myntassets.com/v1/images/style/p...,46694.jpg,44097


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44099 entries, 0 to 44098
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  44099 non-null  int64 
 1   gender              44099 non-null  object
 2   masterCategory      44099 non-null  object
 3   subCategory         44099 non-null  object
 4   articleType         44099 non-null  object
 5   baseColour          44099 non-null  object
 6   season              44099 non-null  object
 7   year                44099 non-null  int64 
 8   usage               44099 non-null  object
 9   productDisplayName  44099 non-null  object
 10  image               44099 non-null  object
 11  filename            44099 non-null  object
 12  index               44099 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 4.4+ MB


In [10]:
selected_features = ['articleType', 'season', 'usage', 'productDisplayName']

In [11]:
selected_features

['articleType', 'season', 'usage', 'productDisplayName']

In [12]:
# Merge Features
combined_features = df['articleType']+' '+df['season']+' '+df['usage']+' '+df['productDisplayName']

In [13]:
# Remove Stop Words
def remove_stopwords(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if not token.is_stop])

In [14]:
def clean_text(text):
    #print(f'Text before Cleaning: {text}')
    # Text to lowercase
    text = text.lower()
    # Remove Numbers from text
    text = re.sub(r'\d+', '', text)
    # Remove Punctuations
    text = re.sub(r'[^\w\s]','', text) 
    #text = remove_punct(text)
    # Remove StopWords
    text = remove_stopwords(text)
    # Remove empty spaces
    text = text.strip()
    # \s+ to match all whitespaces
    # replace them using single space " "
    text = re.sub(r"\s+", " ", text)    
    #print(f'Text after Cleaning: {text}')
    return text

In [15]:
def preprocess_text(df):
    print("Datapreprocessing Started!!!!")
    merged_data = df.apply(lambda x: clean_text(x))
    print("Datapreprocessing Done!!!")
    return merged_data

In [16]:
f = preprocess_text(combined_features)

Datapreprocessing Started!!!!
Datapreprocessing Done!!!


In [17]:
f

0        shirts fall casual turtle check men navy blue ...
1        jeans summer casual peter england men party bl...
2           watches winter casual titan women silver watch
3        track pants fall casual manchester united men ...
4               tshirts summer casual puma men grey tshirt
                               ...                        
44094    casual shoes summer casual gas men caddy casua...
44095    flip flops summer casual lotto mens soccer tra...
44096    tshirts fall casual puma men graphic stellar b...
44097    perfume body mist spring casual rasasi women b...
44098    watches winter casual fossil women pink dial c...
Length: 44099, dtype: object

In [18]:
vectorizer = TfidfVectorizer()

In [19]:
feature_vectors = vectorizer.fit_transform(f)

In [64]:
# Similarity Score
similarity = cosine_similarity(feature_vectors)

In [65]:
print(similarity)

[[1.         0.05123779 0.03316254 ... 0.0072727  0.         0.0307245 ]
 [0.05123779 1.         0.06328849 ... 0.         0.03502307 0.0244758 ]
 [0.03316254 0.06328849 1.         ... 0.0047393  0.01867658 0.02002183]
 ...
 [0.0072727  0.         0.0047393  ... 1.         0.01217987 0.00868074]
 [0.         0.03502307 0.01867658 ... 0.01217987 1.         0.        ]
 [0.0307245  0.0244758  0.02002183 ... 0.00868074 0.         1.        ]]


In [66]:
print(similarity.shape)

(10000, 10000)


In [90]:
# Getting the product name from the user
product_name = 'black shoes sports'

In [91]:
product_name

'adidas mens tshirt'

In [92]:
list_of_all_name = df['productDisplayName'].tolist()

In [93]:
list_of_all_name

['Enamor Women White and Pink Briefs',
 'Aurelia Women Black Printed Kurta',
 'Femella Women Black & Beige Shrug',
 'Nike Women Solid Grey Track Pants',
 'Vishudh Red Leggings',
 'ADIDAS Unisex Blue Duffle Bag',
 'Murcia Women Hths Purple Handbags',
 'Catwalk Women Black Wedges',
 'Crocs Women Olivia Black Sandal',
 'Nike Women Blue Polo T-shirt',
 'Nyk Women Ink Blue Albedo Handbag',
 'French Connection Men Black Belt',
 "Myntra Men's Friend White T-shirt",
 'Catwalk Women Multi Flats',
 'U.S. Polo Assn. Men Solid Ivory Polo Tshirt',
 'Fossil Men Olive Wallet',
 'American Tourister Unisex Blue Backpack',
 'Gini and Jony Girls Pink Skirt',
 'Chimp Men Say Cheese Green Tshirts',
 'ADIDAS Men Navy Blue Short',
 'Wrangler Men Grey Cap',
 'Casio Enticer Men White Dial Multi-Dial Analogue Watch MTP-1192E-7ADF(A166)',
 'Gini and Jony Girls Yellow Top',
 'Chromozome Men Black Briefs',
 'Red Rose Blue & Black Nightdress',
 'Gini and Jony Girls Printed Yellow Top',
 'Nike Unisex Nike Black plai

In [94]:
find_close_math = difflib.get_close_matches(product_name, list_of_all_name)

In [95]:
print(find_close_math)

['Probase Mens  T-shirt', 'Adidas Men Blue Polo T-shirt', 'Fila Men White T-shirt']


In [96]:
close_match = find_close_math[0]
close_match

'Probase Mens  T-shirt'

In [97]:
# Finding the index of product with name
index_product = df[df.productDisplayName == close_match]['index'].values[0]

In [98]:
index_product

7642

In [99]:
# Printing a list of similar products
similarity_score = list(enumerate(similarity[index_product]))

In [100]:
# Sort products based on their similarity score
sorted_similar = sorted(similarity_score, key = lambda x:x[1], reverse=True)

In [101]:
sorted_similar

[(7642, 1.0000000000000002),
 (9338, 0.9233630278183516),
 (9467, 0.9125417921453916),
 (5162, 0.787678657121325),
 (5292, 0.7329829855888271),
 (2023, 0.6916561934706877),
 (8066, 0.6896014929564688),
 (9013, 0.6734740411266381),
 (4000, 0.6725428351181186),
 (5343, 0.6627218964553306),
 (5246, 0.6623777876089972),
 (629, 0.6592222867660111),
 (3818, 0.6579012855725055),
 (8166, 0.6557089311871858),
 (2018, 0.6420386990203473),
 (3078, 0.638773482000471),
 (6457, 0.6283723951475068),
 (2720, 0.6278507447351046),
 (4654, 0.6278507447351046),
 (1589, 0.6230897299097174),
 (6328, 0.6230897299097174),
 (6993, 0.6069456835934871),
 (3420, 0.5882231443463055),
 (9770, 0.5882231443463055),
 (989, 0.5823408544938349),
 (4162, 0.5763925217009258),
 (8592, 0.5740744350075813),
 (3716, 0.5717293348637683),
 (2576, 0.5588087577090592),
 (3771, 0.5574371955844161),
 (2927, 0.5543086784353005),
 (769, 0.5285412200750397),
 (3943, 0.5191075802361765),
 (2134, 0.5130263833976476),
 (1711, 0.490912692

In [102]:
# Print similar products
print('Products suggested for you : \n')
i = 1

for prod in sorted_similar:
    index = prod[0]
    title_from_index = df[df.index==index]['productDisplayName'].values[0]
    if (i<10):
        print(i, '.', title_from_index)
        i+=1

Products suggested for you : 

1 . Probase Mens  T-shirt
2 . Probase Men's 8 Brown T-shirt
3 . Probase Men's 8 Green T-shirt
4 . Probase Men's Graphic Black T-shirt
5 . Probase Men Purple & Blue T-shirt
6 . Probase Men's Spirit Blue T-shirt
7 . Probase Men's Spirit White T-shirt
8 . Probase Men's Mirror On The Brown T-shirt
9 . Probase Men's Mirror On The Grey T-shirt
